In [3]:
# basic imports
import pandas as pd

all_stats = pd.DataFrame({})

for year in range(2003, 2025):
    if year in [2020, 2021]:
        continue
    # load the previously generated csv files

    df_tour_stats = pd.read_csv(f"stats/{year}_tournament_stats.csv")
    df_stats = pd.read_csv(f"stats/{year}_total_stats.csv")
    df_records = pd.read_csv(f"records/{year}_records.csv")
    
    # which cols do we want to substract the tour from the total stats
    cols = ['pts', 'fgm', 'fga', '3pm',
            '3pa', 'ftm', 'fta', 'orb',
            'drb', 'reb', 'ast', 'stl',
            'blk', 'tov', 'pf']
    
    # merge the total and tour stats, then substract the stat columns
    df_stats = pd.merge(df_stats, df_tour_stats, on=["year", "team"], how="left", suffixes=["_total", "_tour"])
    for col in cols:
        total = f"{col}_total"
        tour = f"{col}_tour"
        df_stats[col] = df_stats[total] - df_stats[tour]
        df_stats = df_stats.drop([total, tour], axis=1)
    
    # merge in the records and drop na
    df_stats = pd.merge(df_stats, df_records, on=["year", "team"], how="left")
    df_stats = df_stats.dropna()
    
    # total number of games played
    num_games = df_stats["wins"] + df_stats["losses"]
    
    # columns to be averaged per game
    total_cols = ['pts', 'orb', 'drb', 'reb', 'ast', 'stl', 'blk', 'tov', 'pf']
    
    # append pg to the column names above
    per_game_cols = [col + "pg" for col in total_cols]
    for stat, stat_per_game in zip(total_cols, per_game_cols):
        # add the per game columns to the dataframe
        df_stats.insert(df_stats.columns.get_loc(stat)+1, stat_per_game, df_stats[stat]/num_games)
    df_stats.insert(df_stats.columns.get_loc("losses")+1, "winp", df_stats["wins"]/num_games)
    
    # insert the conference win percentage column
    num_cgames = df_stats["cwins"] + df_stats["closses"]
    df_stats.insert(df_stats.columns.get_loc("closses")+1, "cwinp", df_stats["cwins"]/num_cgames)
    
    # insert the field goal percentage column
    df_stats.insert(df_stats.columns.get_loc("fga")+1, "fgp", df_stats["fgm"]/df_stats["fga"])
    
    # insert the 3 point percentage column
    df_stats.insert(df_stats.columns.get_loc("3pa")+1, "3pp", df_stats["3pm"]/df_stats["3pa"])
    
    # insert the free throw percentage column
    df_stats.insert(df_stats.columns.get_loc("fta")+1, "ftp", df_stats["ftm"]/df_stats["fta"])
    
    # round everything in the dataframe to 3 decimal places
    df_stats = df_stats.round(3)
    
    # append the current year to the list of all years
    all_stats = pd.concat([all_stats, df_stats], ignore_index=True) 
    
all_stats.to_csv("stats/all_stats.csv", index=False)


/tmp/ipykernel_270350/1806513262.py:63: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_stats = pd.concat([all_stats, df_stats], ignore_index=True)


In [4]:
all_stats

,year,team,pts,ptspg,fgm,fga,fgp,3pm,3pa,3pp,...,tovpg,pf,pfpg,seed,wins,losses,winp,cwins,closses,cwinp
0,2004,UConn,2610.0,79.091,994.0,2066.0,0.481,211.0,527.0,0.400,...,13.636,530.0,16.061,2.0,28.0,5.0,0.848,12.0,4.0,0.750
1,2004,Duke,2552.0,79.750,882.0,1869.0,0.472,230.0,626.0,0.367,...,13.469,599.0,18.719,1.0,26.0,6.0,0.812,13.0,3.0,0.812
2,2004,Georgia Tech,2498.0,78.062,887.0,1868.0,0.475,232.0,620.0,0.374,...,14.875,679.0,21.219,3.0,24.0,8.0,0.750,9.0,7.0,0.562
3,2004,Murray State,2677.0,81.121,977.0,1959.0,0.499,223.0,649.0,0.344,...,13.909,600.0,18.182,12.0,28.0,5.0,0.848,14.0,2.0,0.875
4,2004,Oklahoma State,2361.0,78.700,875.0,1703.0,0.514,147.0,383.0,0.384,...,13.200,576.0,19.200,2.0,27.0,3.0,0.900,14.0,2.0,0.875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1265,2024,Dayton,2317.0,74.742,786.0,1647.0,0.477,295.0,734.0,0.402,...,9.710,416.0,13.419,7.0,24.0,7.0,0.774,14.0,4.0,0.778
1266,2024,Grambling State,2298.0,67.588,807.0,1834.0,0.440,178.0,525.0,0.339,...,12.706,544.0,16.000,16.0,21.0,13.0,0.618,14.0,4.0,0.778
1267,2024,Virginia,2098.0,63.576,794.0,1831.0,0.434,217.0,598.0,0.363,...,7.697,445.0,13.485,10.0,23.0,10.0,0.697,13.0,7.0,0.650
1268,2024,Saint Peter's,2091.0,65.344,704.0,1775.0,0.397,188.0,566.0,0.332,...,11.500,618.0,19.312,15.0,19.0,13.0,0.594,12.0,8.0,0.600
